# Multiple Variables stored in one Column

This notebook shows how multiple variables stored in the same column can be isolated.

## "Housekeeping"

In [1]:
% load_ext watermark
% watermark -d -t -v -z -p numpy,pandas

2018-08-26 11:50:39 CEST

CPython 3.6.5
IPython 6.5.0

numpy 1.15.1
pandas 0.23.4


In [2]:
import pandas as pd

## Example: Tuberculosis

### Load the Data

Select the same columns as in the paper and name them accordingly.

In [3]:
columns = ['iso2', 'year',
           'new_sp_m014', 'new_sp_m1524', 'new_sp_m2534', 'new_sp_m3544',
           'new_sp_m4554', 'new_sp_m5564', 'new_sp_m65', 'new_sp_mu',
           'new_sp_f014', 'new_sp_f1524', 'new_sp_f2534', 'new_sp_f3544',
           'new_sp_f4554', 'new_sp_f5564', 'new_sp_f65', 'new_sp_fu']
tb = pd.read_csv('data/tb.csv', usecols=columns)

rename = {c: c[7:] for c in columns if c.startswith('new_sp_')}
rename = {'iso2': 'country', **rename}
tb = tb.rename(columns=rename)

### Messy Data

The data are assumed to be provided as below. Except for the *country* and *year* columns, the remaining columns are actually joint realizations of two variables **sex** and **age**.

In [4]:
tb[(tb['year'] == 2000)].head(10)

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
10,AD,2000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,AE,2000,2.0,4.0,4.0,6.0,5.0,12.0,10.0,NaN,3.0,16.0,1.0,3.0,0.0,0.0,4.0,NaN
60,AF,2000,52.0,228.0,183.0,149.0,129.0,94.0,80.0,NaN,93.0,414.0,565.0,339.0,205.0,99.0,36.0,NaN
87,AG,2000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN
136,AL,2000,2.0,19.0,21.0,14.0,24.0,19.0,16.0,NaN,3.0,11.0,10.0,8.0,8.0,5.0,11.0,NaN
165,AM,2000,2.0,152.0,130.0,131.0,63.0,26.0,21.0,NaN,1.0,24.0,27.0,24.0,8.0,8.0,4.0,NaN
178,AN,2000,0.0,0.0,1.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN
207,AO,2000,186.0,999.0,1003.0,912.0,482.0,312.0,194.0,NaN,247.0,1142.0,1091.0,844.0,417.0,200.0,120.0,NaN
236,AR,2000,97.0,278.0,594.0,402.0,419.0,368.0,330.0,NaN,121.0,544.0,479.0,262.0,230.0,179.0,216.0,NaN
265,AS,2000,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


### Molten Data

As in the previous notebook the [*pd.melt*](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) function can be used to un-pivot the columns. As before, pandas keeps rows for columns with missing data that are then discarded (then, without any more missing values, the column's data type is casted as integer). Furthermore, the resulting *molten* dataset is sorted as in the paper.

In [5]:
molten_tb = pd.melt(tb, id_vars=['country', 'year'], var_name='column', value_name='cases')
molten_tb = molten_tb[molten_tb['cases'].notnull()]
molten_tb['cases'] = molten_tb['cases'].astype(int)
molten_tb = molten_tb.sort_values(['country', 'year', 'column']).reset_index(drop=True)

In [6]:
molten_tb[(molten_tb['year'] == 2000)].head(10)

,country,year,column,cases
49,AD,2000,m014,0
50,AD,2000,m1524,0
51,AD,2000,m2534,1
52,AD,2000,m3544,0
53,AD,2000,m4554,0
54,AD,2000,m5564,0
55,AD,2000,m65,0
165,AE,2000,f014,3
166,AE,2000,f1524,16
167,AE,2000,f2534,1


### Tidy Data

Using the [*pd.Series.str.extract*](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html) method the two variables are isolated. The age labels are renamed as in the paper.

In [7]:
tidy_tb = molten_tb[['country', 'year', 'cases']]
tidy_tb[['sex', 'age']] = molten_tb['column'].str.extract(r'(f|m)(.*)')
tidy_tb['age'] = tidy_tb['age'].map({
    '014': '0-14', '1524': '15-24', '2534': '25-34',
    '3544': '35-44', '4554': '45-54', '5564': '55-64',
    '65': '65+', 'u': 'unknown'
})
tidy_tb = tidy_tb[['country', 'year', 'sex', 'age', 'cases']]

In [8]:
tidy_tb[(tidy_tb['year'] == 2000)].head(10)

,country,year,sex,age,cases
49,AD,2000,m,0-14,0
50,AD,2000,m,15-24,0
51,AD,2000,m,25-34,1
52,AD,2000,m,35-44,0
53,AD,2000,m,45-54,0
54,AD,2000,m,55-64,0
55,AD,2000,m,65+,0
165,AE,2000,f,0-14,3
166,AE,2000,f,15-24,16
167,AE,2000,f,25-34,1
